In [3]:
#!conda list

# packages in environment at F:\anaconda3\envs\BirdClassifier:
#
# Name                    Version                   Build  Channel
argon2-cffi               20.1.0           py39h2bbff1b_1  
async_generator           1.10               pyhd3eb1b0_0  
attrs                     21.2.0             pyhd3eb1b0_0  
azure-cognitiveservices-vision-customvision 3.1.0                    pypi_0    pypi
azure-common              1.1.27                   pypi_0    pypi
backcall                  0.2.0              pyhd3eb1b0_0  
bleach                    3.3.1              pyhd3eb1b0_0  
ca-certificates           2021.7.5             haa95532_1  
certifi                   2021.5.30        py39haa95532_0  
cffi                      1.14.6           py39h2bbff1b_0  
charset-normalizer        2.0.3                    pypi_0    pypi
colorama                  0.4.4              pyhd3eb1b0_0  
decorator                 5.0.9              pyhd3eb1b0_0  
defusedxml                0.7.1              pyhd3eb

In [2]:
#!pip install azure-cognitiveservices-vision-customvision

  Using cached azure_cognitiveservices_vision_customvision-3.1.0-py2.py3-none-any.whl (62 kB)
  Using cached msrest-0.6.21-py2.py3-none-any.whl (85 kB)
  Using cached azure_common-1.1.27-py2.py3-none-any.whl (12 kB)
  Using cached isodate-0.6.0-py2.py3-none-any.whl (45 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Using cached oauthlib-3.1.1-py2.py3-none-any.whl (146 kB)


In [10]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch
from msrest.authentication import ApiKeyCredentials 
import numpy as np

In [45]:
ENDPOINT = "https://customvisionbirdclassifier.cognitiveservices.azure.com/"

# Replace with a valid key
training_key = "9256a1fc93684a298b0a43f0622a5b0d"
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
publish_iteration_name = "classifyBirdModel"

trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Bird Classification")

print("Project created!")

Creating project...
Project created!


In [14]:
import os
os.chdir('./bird-photos/custom_photos')

In [46]:
# creating a list of bird types (they are the folder names)
tags = [name for name in os.listdir('.') if os.path.isdir(name)]
print(tags)

['American Crow', 'American Goldfinch (Female)', 'American Goldfinch (Male)', 'American Robin (Adult)', 'American Robin (Juvenile)', 'Blue Jay', 'Common Grackle', 'House Sparrow (Female)', 'House Sparrow (Male)', 'House Wren', 'Mourning Dove', 'Northern Cardinal (Adult Male)', 'Northern Cardinal (Female)', 'Red-tailed Hawk (Dark morph)', 'Red-tailed Hawk (Light morph immature)', 'Tufted Titmouse']


In [47]:
#createTag function
def createTag(tag):
    result = trainer.create_tag(project.id, tag)
    print(f'{tag} created with id: {result}')
    return result.id

In [48]:
# createImageList function
def createImageList(tag, tag_id):
# Set directory to current tag.
    base_image_url = f"./{tag}/"
    photo_name_list = os.listdir(base_image_url)
    image_list = []
    for file_name in photo_name_list:
        with open(base_image_url+file_name, "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=base_image_url+file_name, contents=image_contents.read(), tag_ids=[tag_id]))
    return image_list

In [49]:
#uploadImageList
def uploadImageList(image_list):
    upload_result = trainer.create_images_from_files(project_id=project.id, batch=image_list)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)

In [52]:
# using functions written

for tag in tags: 
    tag_id = createTag(tag)
    print(f"tag creation done with tag id {tag_id}")
    image_list = createImageList(tag, tag_id)
    print("image_list created with length " + str(len(image_list)))
    # Break list into lists of 25 and upload in batches.
    for i in range(0, len(image_list), 25):
        batch = ImageFileCreateBatch(images=image_list[i:i + 25])
        print(f'Upload started for batch {i} total items {len  (image_list)} for tag {tag}...')
        uploadImageList(batch)
        print(f"Batch {i} Image upload completed. Total uploaded  {len(image_list)} for tag {tag}")

American Crow created with id: {'additional_properties': {}, 'id': 'a809dcfa-fd31-48b9-a1d8-778416c2a8cf', 'name': 'American Crow', 'description': None, 'type': 'Regular', 'image_count': 0}
tag creation done with tag id a809dcfa-fd31-48b9-a1d8-778416c2a8cf
image_list created with length 117
Upload started for batch 0 total items 117 for tag American Crow...
Batch 0 Image upload completed. Total uploaded  117 for tag American Crow
Upload started for batch 25 total items 117 for tag American Crow...
Batch 25 Image upload completed. Total uploaded  117 for tag American Crow
Upload started for batch 50 total items 117 for tag American Crow...
Batch 50 Image upload completed. Total uploaded  117 for tag American Crow
Upload started for batch 75 total items 117 for tag American Crow...
Batch 75 Image upload completed. Total uploaded  117 for tag American Crow
Upload started for batch 100 total items 117 for tag American Crow...
Batch 100 Image upload completed. Total uploaded  117 for tag Am

Batch 50 Image upload completed. Total uploaded  117 for tag House Sparrow (Female)
Upload started for batch 75 total items 117 for tag House Sparrow (Female)...
Batch 75 Image upload completed. Total uploaded  117 for tag House Sparrow (Female)
Upload started for batch 100 total items 117 for tag House Sparrow (Female)...
Batch 100 Image upload completed. Total uploaded  117 for tag House Sparrow (Female)
House Sparrow (Male) created with id: {'additional_properties': {}, 'id': 'e510dc4a-56a4-4d0a-a862-0bb2fd88d579', 'name': 'House Sparrow (Male)', 'description': None, 'type': 'Regular', 'image_count': 0}
tag creation done with tag id e510dc4a-56a4-4d0a-a862-0bb2fd88d579
image_list created with length 119
Upload started for batch 0 total items 119 for tag House Sparrow (Male)...
Batch 0 Image upload completed. Total uploaded  119 for tag House Sparrow (Male)
Upload started for batch 25 total items 119 for tag House Sparrow (Male)...
Batch 25 Image upload completed. Total uploaded  119

Batch 0 Image upload completed. Total uploaded  120 for tag Tufted Titmouse
Upload started for batch 25 total items 120 for tag Tufted Titmouse...
Batch 25 Image upload completed. Total uploaded  120 for tag Tufted Titmouse
Upload started for batch 50 total items 120 for tag Tufted Titmouse...
Batch 50 Image upload completed. Total uploaded  120 for tag Tufted Titmouse
Upload started for batch 75 total items 120 for tag Tufted Titmouse...
Batch 75 Image upload completed. Total uploaded  120 for tag Tufted Titmouse
Upload started for batch 100 total items 120 for tag Tufted Titmouse...
Batch 100 Image upload completed. Total uploaded  120 for tag Tufted Titmouse
